In [2]:
import requests
import time
import json

def get_json(url):
    r_ = requests.get(url)
    try:
        return r_.json()
    except ValueError:
        time.sleep(1)
        return get_json(url)

user_key = '1a49911e9216b8037dc430f811fd5b6b'

# Organizations


In [2]:
r = requests.get('https://api.crunchbase.com/v/3/organizations?user_key={}'.format(user_key))

In [3]:
r.json()['data']['paging']

{u'current_page': 1,
 u'items_per_page': 100,
 u'next_page_url': u'https://api.crunchbase.com/v/3/organizations?page=2',
 u'number_of_pages': 4086,
 u'prev_page_url': None,
 u'sort_order': u'created_at DESC',
 u'total_items': 408579}

In [4]:
r.json()['data']['items'][0]

{u'properties': {u'api_path': u'organizations/africa-consulting-trading',
  u'city_name': None,
  u'country_code': None,
  u'created_at': 1448335501,
  u'domain': u'act-afrique.com',
  u'facebook_url': None,
  u'homepage_url': u'http://www.act-afrique.com',
  u'linkedin_url': None,
  u'name': u'Africa Consulting & Trading',
  u'permalink': u'africa-consulting-trading',
  u'primary_role': u'company',
  u'profile_image_url': u'http://public.crunchbase.com/t_api_images/v1448335489/xdrsfaiuew98lphqhmlo.png',
  u'region_name': None,
  u'short_description': u'ACT is the culmination of a long journey marked by the passion of its founder and its partners for Africa and its development.',
  u'twitter_url': None,
  u'updated_at': 1448335502,
  u'web_path': u'organization/africa-consulting-trading'},
 u'type': u'OrganizationSummary',
 u'uuid': u'5bca314aef44c11c0e51c394c05330f6'}

In [5]:
organizations = r.json()['data']['items']

In [15]:
len(organizations)

72000

Script to scrap all the organizations, took arount 5 hours (because of the API bandwith)

In [14]:
organizations = []
for i in xrange(1, 4087):
    link = 'https://api.crunchbase.com/v/3/organizations?page={}&user_key={}'.format(i, user_key)
    if i%100 == 0:
        print 'page ', i, ' length is ',len(organizations)
    r_json = get_json(link)
    organizations += r_json['data']['items']
    time.sleep(0.1)

with open('tmp/organizations_json.json', 'w') as f:
    json.dump(organizations, f)

page  3400  length is  3300
page  3500  length is  13300
page  3600  length is  23300
page  3700  length is  33300
page  3800  length is  43300
page  3900  length is  53300
page  4000  length is  63300


In [218]:
# map from the permalink of a company to its index in the list organizations_tot
permalink2index = {organizations_tot[i]['properties']['permalink']: i for i in xrange(len(organizations_tot))}

In [219]:
permalink2index

{u'd-n-laoghaire-institute-of-art-design-and-technology': 173561,
 u'gmc-truck': 70047,
 u'mdbg': 310707,
 u'hoyer-associates': 190470,
 u'harley-co': 297303,
 u'westrock-trading': 16248,
 u'mdbn': 86550,
 u'digital-rockers': 81687,
 u'bloom-company': 33835,
 u'fantasy-shopper': 336237,
 u'papatv-cn': 48763,
 u'phoenix-rehabilitation-and-health-services': 44445,
 u'napcon-associates': 282300,
 u'paizo': 102319,
 u'paiza': 88443,
 u'talontax': 191624,
 u'savvify': 178490,
 u'searchsense': 161250,
 u'godaam': 308299,
 u'human-rights-foundation': 74716,
 u'artmachine-inc': 280015,
 u'bigbelly-solar': 235125,
 u'dw-healthcare-partners': 297915,
 u'american-farmland': 11984,
 u'bluetree-capital-group': 388513,
 u'reconquistar-editora': 22047,
 u'fundation': 281763,
 u'sleeklighting': 163876,
 u'xodo': 70433,
 u'mayartech': 208825,
 u'ceph-storage': 326240,
 u'american-health-information-management-association': 185980,
 u'dockbit': 38283,
 u'regency-beauty-institute-darien': 170905,
 u'yoga

# Relationships


In [37]:
# We are retrieving only the companies founded before 2011
len(focus_companies)

25171

In [10]:
relationships = ['founders', 'board_members_and_advisors', 'investors', 'headquarters', 'offices', 'categories', 'competitors', 'funding_rounds', 'acquisitions', 'acquired_by', 'ipo']

relationships_infos = {}
relationships_infos['person'] = ['bio', 'born_on', 'permalink']
relationships_infos['address'] = ['city', 'country']
relationships_infos['market'] = ['name']
relationships_infos['product'] = ['name']
relationships_infos['organization'] = ['permalink']
relationships_infos['fundinground'] = ['announced_on', 'funding_type', 'money_raised_usd']
relationships_infos['acquisition'] = ['announced_on']
relationships_infos['ipo'] = ['went_public_on']

Script to extract features for each organizations, took 1min30 to run for 10 organizations.

In [20]:
%%time
organizations_dict = {}
with open('tmp/organizations_dict_to2010.json', 'r') as f:
    organizations_dict = json.load(f)
for i, permalink in enumerate(focus_companies.keys()[len(organizations_dict):25171]):
    if i%50 ==0:
        print 'iteration ', i, ' length is ', len(organizations_dict)
    infos = {}
    #permalink = organization['properties']['permalink']
    for relation in relationships:
        time.sleep(0.1)
        url = u'https://api.crunchbase.com/v/3/organizations/{}/{}?user_key={}'.format(permalink, relation, user_key)
        r_json = get_json(url)
        if 'data' not in r_json:
            continue
        if 'items' in r_json['data']:
            items = r_json['data']['items']
        elif 'item' in r_json['data']:
            items = r_json['data']['item']
        else:
            items = []
        number_pages = r_json['data']['paging']['number_of_pages']
        next_page_url = r_json['data']['paging']['next_page_url']
        for j in xrange(2, number_pages + 1):
            url = next_page_url[:-1]+str(j)+'&user_key='+user_key
            r_json = get_json(url)
            items += r_json['data']['items']
        
        relation_items = []
        if type(items) is dict:
            items = [items]
        for item in items:
            if 'relationships' in item:
                for relationships_key in item['relationships']:
                    relationships_key = relationships_key.lower()
                    if relationships_key in relationships_infos:
                        item_temp = {}
                        for infos_key in relationships_infos[relationships_key]:
                            item_temp[infos_key] = item['relationships'][relationships_key]['properties'][infos_key]
                        relation_items.append(item_temp)
            relationships_key = item['type'].lower()
            if relationships_key in relationships_infos:
                item_temp = {}
                for infos_key in relationships_infos[relationships_key]:
                    item_temp[infos_key] = item['properties'][infos_key]
                relation_items.append(item_temp)
        infos[relation] = relation_items
    organizations_dict[permalink] = infos

    with open('tmp/organizations_dict_to2010.json', 'w') as f:
        json.dump(organizations_dict, f)

iteration  0  length is  2000


KeyboardInterrupt: 

In [52]:
with open('tmp/organizations_total_json.json', 'r') as f:
    organizations_tot = json.load(f)

In [65]:
organizations_tot[permalink2index['express-inn-hotel']]['properties']['short_description']

u'Hotel opportunity: Selling 50% equity for $1M, Operating & managing partners takes no salary, strictly profit & equity driven.'

In [64]:
permalink2index = {v['properties']['permalink']:i for i, v in enumerate(organizations_tot)}
print len(permalink2index)
print len(organizations_tot)

407691
408422


In [69]:
# Adding the short description to the stored information
for o in organizations_dict:
    if o not in permalink2index:
        i = permalink2index[o]
        organizations_dict[o]['short_description'] = organizations_tot[i]['properties']['short_description']


here
here
here
here
here
here
here
here
here
here


In [68]:
# Adding the foundation date to the stored information
for o in organizations_dict:
    if o not in organizations_xlsx:
        organizations_dict[o]['month_founded'] = organizations_xlsx[o]


In [67]:
with open('tmp/organizations_dict_to2010.json', 'w') as f:
    json.dump(organizations_dict, f)

# Retrieving the information from the Excel dataset

In [7]:
from openpyxl import load_workbook
wb = load_workbook(filename='test.xlsx', read_only=True)

/usr/local/lib/python2.7/site-packages/openpyxl/workbook/names/named_range.py:124: UserWarning: Discarded range with reserved name
  warnings.warn("Discarded range with reserved name")


In [8]:
wb.get_sheet_names()

[u'License',
 u'Analysis',
 u'Companies',
 u'Rounds',
 u'Investments',
 u'Acquisitions',
 u'Additions']

In [9]:
wc = wb['Companies']

In [12]:
# Finding the column of the permalink and the foundation date
rows = enumerate(wc.rows)
row = rows.next()
for i, val in enumerate(row[1]):
    print i,' ',val.value

0   permalink
1   name
2   homepage_url
3   category_list
4   market
5   funding_total_usd
6   status
7   country_code
8   state_code
9   region
10   city
11   funding_rounds
12   founded_at
13   founded_month
14   founded_quarter
15   founded_year
16   first_funding_at
17   last_funding_at


In [18]:
# Dictionary with (key, value) = (permalink, founded month)
organizations_xlsx = {}

rows = enumerate(wc.rows)

# Retrieving the header
row = rows.next()

for row in rows:
    # Get rid of the first 14 characters which are '/organizations/'
    organizations_xlsx[row[1][0].value[14:]] = row[1][13].value
organizations_xlsx

{u'pocket-video': None,
 u'caetus': u'2015-01',
 u'springpath-inc': u'2012-05',
 u'digipath': None,
 u'mdbn': u'2013-06',
 u'fantasy-shopper': u'2010-11',
 u'papatv-cn': u'2014-11',
 u'choose-digital': u'2011-05',
 u'aboutmystar': u'2011-09',
 u'fiture': None,
 u'glamorous-travel': None,
 u'docusys': None,
 u'veebow': u'2011-01',
 u'savvify': None,
 u'linquet': u'2012-01',
 u'prepmatic': u'2011-01',
 u'dreambox-learning': u'2004-06',
 u'social-tools': u'2012-05',
 u'shawarmanji': u'2012-01',
 u'vaxin-inc': u'1997-01',
 u'scoot-doodle': u'2012-01',
 u'procurics': u'2012-12',
 u'nichewith': u'2013-01',
 u'acylin-therapeutics': u'2009-01',
 u'nanopharmaceuticals': u'2013-01',
 u'boston-university': u'2014-01',
 u'your-style-unzipped': u'2014-01',
 u'fundation': u'2011-11',
 u'crossbar': u'2008-01',
 u'cytochroma': None,
 u'smarthabitat': u'2010-01',
 u'picovico': u'2011-04',
 u'rebounces': u'2008-09',
 u'theron-pharmaceuticals': None,
 u'organic-merchants': u'2013-09',
 u'thoughtbubble-pr

In [19]:
len(organizations_xlsx)

61398

In [ ]:
with open('tmp/organizations_xlsx.json', 'w') as f:
    json.dump(organizations_xlsx, f)

In [17]:
with open('tmp/organizations_xlsx.json', 'r') as f:
    organizations_xlsx = json.load(f)

In [7]:
# Studying the foundation year
import collections
companies_years = collections.Counter(organizations_xlsx.values())

In [8]:
print 'Number of companies funded before 2011: ',sum([val for k, val in companies_years.iteritems() if k and int(k[:4]) < 2011])

Number of companies funded before 2011:  25171


In [9]:
# Retrieving the list of the company founded before 2011
focus_companies = {k:v for k,v in organizations_xlsx.iteritems() if v and int(v[:4]) < 2011}
len(focus_companies)

25171

# Summary of the scrapping

First, we scrapped all organizations name and short description from the REST API in the list 'organizations_tot', which gave 408 422 organizations and took 3h30. We had to sleep in between our API calls because the server could identify our API key and limit our call. This file is saved as a json in 'tmp/organizations_json.json', we mapped the company permalink (ie name in the REST API naming) with its index in the previous list for an easy access.

Second, we had to retrieve the features we wanted from the REST API. The API is built around the relationships between the different instance (company, person, funding...) so we went through the different relationships and instance to store only the one we care about. This provided the list 'relationships', which stores the relationships we consider, and the dictionary 'relationships_infos' which stores for each instance type the attributes we consider. Once this preliminary work has been done, we wrote the API calls to store these attributes for each organizations. Trying it with our big list previously retrieved, we realized there were too many missing values and it would take too much time. As a result, we used the excel API to retrieve a list of companies name with at least some information. The excel API provides a list of 61 399 companies, with its categories, offices locations and funding informations (but there is nothing about the founders, the competitors, the board members...). Nonetheless, some information may be retrieved from the excel data as the locations and the funding, maybe we don't need to call the api for that (I didn't try it). This list is stored in the file 'organizations_xlsx.json'

Third, we focused on the companies provided by the excel list and applied them the API calls routine to retrieve the different featurs. The first file contains 1000 companies and took 3 hours to run, the json is stored in 'organizations_dict.json'. It's a dictionary by permalink with attributes corresponding to the features we care about (some atributes are also dictionaries). 

### Next step

We need to write a routine to clean the features of the data: to interpret the text values (in the bio of the persons and in the short description of the company) which may contain relevant information, to decide what to do with missing data (maybe we could infer them based on simple averaging methods), to run first models to get a sense of which features are meaningful.
Based on these insights we would be able to refine our features scrapping method and run it for more companies.